In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [ ]:
data = pd.read_csv('dataset/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [90]:
data = data.iloc[:, 3:]
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [92]:
data.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
Exited                2
dtype: int64

In [93]:
gender_label_encoder = LabelEncoder()
data['Gender'] = gender_label_encoder.fit_transform(data['Gender'])

geo_oh_encoder = OneHotEncoder(sparse_output=False)
geo_encoded = geo_oh_encoder.fit_transform(data[['Geography']])

geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_oh_encoder.get_feature_names_out(['Geography']))
data = pd.concat([data, geo_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)

In [94]:
X = data.drop(['Exited'], axis = 1)
y = data['Exited']

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
with open('artifacts/gender_label_encoder.pkl', 'wb') as file:
    pickle.dump(gender_label_encoder, file)

with open('artifacts/geo_oh_encoder.pkl', 'wb') as file:
    pickle.dump(geo_oh_encoder, file)

with open('artifacts/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [101]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import datetime

In [99]:
model = Sequential([
    Dense(64, activation='ReLU', input_shape=(X_train.shape[1], )),
    Dense(32, activation='ReLU'),
    Dense(1, activation='sigmoid')
]
)

In [100]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                832       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [104]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [109]:
log_dir = 'logs/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tf_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [110]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=100, 
                    callbacks=[tf_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3312 - accuracy: 0.8626 - val_loss: 0.3443 - val_accuracy: 0.8630
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3301 - accuracy: 0.8625 - val_loss: 0.3601 - val_accuracy: 0.8565
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3278 - accuracy: 0.8661 - val_loss: 0.3496 - val_accuracy: 0.8555
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3252 - accuracy: 0.8651 - val_loss: 0.3529 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3218 - accuracy: 0.8660 - val_loss: 0.3505 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3193 - accuracy: 0.8677 - val_loss: 0.3484 - val_accuracy: 0.8615
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3199 - accuracy: 0.8692 - val_loss: 0.3546 - val_accuracy: 0.8615

In [111]:
model.save('artifacts/model.h5')

d:\ML\Churn\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [112]:
%load_ext tensorboard

In [116]:
%tensorboard --logdir logs/fit20241223-125403

Reusing TensorBoard on port 6007 (pid 7544), started 0:00:04 ago. (Use '!kill 7544' to kill it.)

# Testing

In [117]:
import numpy as np
from tensorflow.keras.models import load_model

In [118]:
model = load_model('model.h5')

with open('gender_label_encoder.pkl', 'rb') as file:
    gender_label_encoder = pickle.load(file)
with open('geo_oh_encoder.pkl', 'rb') as file:
    geo_oh_encoder = pickle.load(file)
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)